# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 0.9920361896423422
 1.8359411737135616
 2.0263348709525957

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0         0.0         0.0
 0.603325    1.0         0.0
 0.00278082  0.00140112  1.0
U factor:
3×3 Matrix{Float64}:
 0.673499  0.44472   0.717722
 0.0       0.721092  0.197573
 0.0       0.0       0.985644

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [9]:
Alu

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0         0.0         0.0
 0.603325    1.0         0.0
 0.00278082  0.00140112  1.0
U factor:
3×3 Matrix{Float64}:
 0.673499  0.44472   0.717722
 0.0       0.721092  0.197573
 0.0       0.0       0.985644

In [4]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0         0.0         0.0
 0.603325    1.0         0.0
 0.00278082  0.00140112  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.673499  0.44472   0.717722
 0.0       0.721092  0.197573
 0.0       0.0       0.985644

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999996
 1.0

In [8]:
Alu\b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999996
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [10]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [22]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.00238103   0.000199832  -0.999997
 -0.856232    -0.516588      0.00193549
 -0.516586     0.856234      0.00140111
R factor:
3×3 Matrix{Float64}:
 -0.786585  -0.8919    -0.942644
  0.0        0.617424   0.169366
  0.0        0.0       -0.985641

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [23]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.00238103   0.000199832  -0.999997
 -0.856232    -0.516588      0.00193549
 -0.516586     0.856234      0.00140111

In [24]:
Aqr.R

3×3 Matrix{Float64}:
 -0.786585  -0.8919    -0.942644
  0.0        0.617424   0.169366
  0.0        0.0       -0.985641

In [25]:
Aqr\b

3-element Vector{Float64}:
 1.0000000000000013
 0.9999999999999989
 1.0000000000000004

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [27]:
A

3×3 Matrix{Float64}:
 0.00187288  0.00224702  0.987916
 0.673499    0.44472     0.717722
 0.406339    0.989403    0.630593

In [28]:
A'

3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.00187288  0.673499  0.406339
 0.00224702  0.44472   0.989403
 0.987916    0.717722  0.630593

In [29]:
A + A'

3×3 Matrix{Float64}:
 0.00374576  0.675747  1.39426
 0.675747    0.889439  1.70712
 1.39426     1.70712   1.26119

In [26]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -1.037812906305095
 -0.25314384662172784
  3.4453275589496206
vectors:
3×3 Matrix{Float64}:
 -0.656573   0.638743  -0.401146
 -0.357897  -0.731984  -0.57975
  0.663944   0.237079  -0.709205

The values and the vectors can be extracted from the Eigen type by special indexing

In [30]:
AsymEig.values

3-element Vector{Float64}:
 -1.037812906305095
 -0.25314384662172784
  3.4453275589496206

In [31]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.656573   0.638743  -0.401146
 -0.357897  -0.731984  -0.57975
  0.663944   0.237079  -0.709205

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [37]:
AsymEig.values' * AsymEig.vectors

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 3.0595  0.339216  -1.88037

In [43]:
(AsymEig.vectors * AsymEig.values)'

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 -0.862374  -1.4407  -3.19251

In [48]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
 1.0          1.94289e-16  -2.63678e-16
 1.55431e-15  1.0           1.55431e-15
 5.55112e-16  7.77156e-16   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [49]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [50]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [56]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-3.1192327422706096

In [57]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [58]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [59]:
@time eigvals(Asym);

  0.191745 seconds (133.94 k allocations: 15.452 MiB, 4.95% gc time, 72.74% compilation time)


In [60]:
@time eigvals(Asym_noisy);

  0.446653 seconds (13 allocations: 7.920 MiB)


In [61]:
@time eigvals(Asym_explicit);

  0.057092 seconds (5.88 k allocations: 8.358 MiB, 9.33% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [62]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.694764 seconds (782.61 k allocations: 226.901 MiB, 14.67% gc time, 29.08% compilation time)


6.388276789655706

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [64]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [65]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 7//10  1//10  3//10
 3//10  1//10  7//10
 1//5   1//10  1//10

In [66]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 11//10
 11//10
  2//5

In [67]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [68]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  0//1  0//1
 2//7  1//1  0//1
 3//7  4//5  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 7//10  1//10   3//10
 0//1   1//14   1//70
 0//1   0//1   14//25

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [69]:
using LinearAlgebra

In [70]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]


5×5 Matrix{Int64}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [73]:
A_eig = eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514494
  542.4677301466137
vectors:
5×5 Matrix{Float64}:
 -0.147575  -0.647178  -0.010882    0.548903   -0.507907
 -0.256795   0.173068  -0.834628   -0.239864   -0.387253
 -0.185537  -0.239762   0.422161   -0.731925   -0.440631
  0.819704   0.247506   0.0273194   0.0366447  -0.514526
 -0.453805   0.657619   0.352577    0.322668   -0.364928

In [74]:
A_eigv = A_eig.values

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514494
  542.4677301466137

In [78]:
@assert A_eigv ≈ [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

In [75]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

LoadError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [91]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [89]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

LoadError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [92]:
A_lowertri = tril(A)

5×5 Matrix{Int64}:
 140    0    0    0   0
  97  106    0    0   0
  74   89  152    0   0
 168  131  144   54   0
 131   36   71  142  36

In [93]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]

In [102]:
λ1, λ2, λ3 = 1//1, 1//2, 1//4
v1, v2, v3 = [1,0,0], [1,1,0], [1,1,1]
V, Λ = [v1 v2 v3], Diagonal([λ1, λ2, λ3])

([1 1 1; 0 1 1; 0 0 1], Rational{Int64}[1//1 0//1 0//1; 0//1 1//2 0//1; 0//1 0//1 1//4])

In [103]:
V

3×3 Matrix{Int64}:
 1  1  1
 0  1  1
 0  0  1

In [106]:
V*Λ

3×3 Matrix{Rational{Int64}}:
 1//1  1//2  1//4
 0//1  1//2  1//4
 0//1  0//1  1//4

In [108]:
A = V*Λ/V

3×3 Matrix{Rational{Int64}}:
 1//1  -1//2  -1//4
 0//1   1//2  -1//4
 0//1   0//1   1//4

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback

Please click on `Validate` on the top, once you are done with the exercises.